In [35]:
import os
import h5py
import argparse
import numpy as np
from os import listdir
import cv2


def convert_file(input_dir, filename, output_dir):
    filepath = input_dir + '/' + filename
    img = cv2.imread(filepath)
    #img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    new_filepath = output_dir + '/' + filename[:-4] + '.h5'
    f = h5py.File(new_filepath,'w')
    #dt = h5py.special_dtype(vlen=np.dtype('uint8'))
    dset = f.create_dataset(filename, (100, 1080, 1920, 3), dtype=np.dtype('uint8'))
    dset[0] = img#[None]


def convert_dir(dir_name, cat):
    input_file_folder = '/home/user/Harshita/Frames/'+cat+'/'+dir_name
    output_file_folder='/home/user/Harshita/Frames/HD5_Files/'+cat+'/'+dir_name
    if not os.path.exists(output_file_folder):
        os.makedirs(output_file_folder)
    for filename in os.listdir(input_file_folder):
        if filename[-4:] == '.jpg':
            convert_file(input_file_folder, filename, output_file_folder)

In [36]:
# DD_folders = [f for f in listdir('/home/user/Harshita/Frames/ASD') ]
# for dir_name in DD_folders:
#     convert_dir(dir_name,'ASD')

In [37]:
# import h5py
# import numpy as np
# from PIL import Image
# filename = '21_02_03_01_DD_Ch1_20210203112600_frame_43.jpg'
# hdf = h5py.File("/home/user/Harshita/Frames/HD5_Files/DD/21_02_03_01_DD/21_02_03_01_DD_Ch1_20210203112600_frame_43.h5",'r')
# print(hdf.keys())
# array = hdf[filename][:][0]

# print(array.shape)
# img = Image.fromarray(array.astype('uint8'), 'RGB')
# print(filename[:-3])
# img.save(filename, "PNG")
# img.show()

### Filtering Training images that have annotations

In [38]:
from torch_snippets import *
from PIL import Image
import xml.etree.ElementTree as ET
from typing import Dict, List
from tqdm import tqdm
import re
import pandas as pd
import os
import shutil

    


In [70]:
def parse_xml_file(objects,filename,invalid_cnt):
    ann_tree = ET.parse(filename)       #'../Annotations/DD/VOC XML/21_01_25_01_DD/21_01_25_01_DD_Ch1_20210125154231_frame_123.xml'
    ann_root = ann_tree.getroot()
    #objects=[]
    size_obj=ann_root.find('size')
    height=float(size_obj.findtext('height'))
    width=float(size_obj.findtext('width'))
    
    for obj in ann_root.findall('object'):
        bb={}
        bb['imageID']=ann_root.findtext('filename')[:-4]
        bb['LabelName']=obj.findtext('name')
        bb_obj=obj.find('bndbox')
        if (int(bb_obj.findtext('xmin')) >= int(bb_obj.findtext('xmax'))) or (int(bb_obj.findtext('ymin')) >= int(bb_obj.findtext('ymax'))):
            print('Invalid count')
            #continue
        bb['XMin']=(int(bb_obj.findtext('xmin')))/width
        bb['XMax']=(int(bb_obj.findtext('xmax'))+1)/width

        bb['YMin']=(int(bb_obj.findtext('ymin')))/height
        bb['YMax']=(int(bb_obj.findtext('ymax'))+1)/height
        bb['Confidence']=1
        
        objects.append(bb)


In [71]:
list_of_folders=['../Annotations/DD/VOC XML/','../Annotations/TD/VOC XML/']
objects=[]
invalid_cnt=0
for folder in list_of_folders:
    videos=[f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder,f))]
    for video in videos:
        full_path=os.path.join(folder,video)
        files = [os.path.join(full_path,f) for f in os.listdir(full_path)]
        for f in files:
            parse_xml_file(objects,f,invalid_cnt)

print(invalid_cnt)

Invalid count

Invalid count

Invalid count

0

In [72]:

df=pd.DataFrame(objects)
print(len(df))
#df.drop(df[df['YMin']>=df['YMax']].index, inplace=True)
#df.drop(df[df['XMin']>=df['XMax']].index, inplace=True)
df.to_csv('df.csv')
print(len(df))

1353

1353

In [73]:

# list_of_folders=['../Frames/DD/','../Frames/TD/']
# objects=[]
# for folder in list_of_folders:
#     videos=[os.path.join(folder,f) for f in os.listdir(folder) if os.path.isdir(os.path.join(folder,f))]
#     for video in videos:
#         for f in os.listdir(video):
#             shutil.copy2(os.path.join(video,f),'../Frames/Training Frames/')


In [74]:
df['Area']=(df['XMax']-df['XMin'])*(df['YMax']-df['YMin'])

### Remove BBs where xmin>=xmax or yMin>=YMax

In [75]:
print(df[df['XMin']>=df['XMax']])
print(df[df['YMin']>=df['YMax']])

Empty DataFrame
Columns: [imageID, LabelName, XMin, XMax, YMin, YMax, Confidence, Area]
Index: []

Empty DataFrame
Columns: [imageID, LabelName, XMin, XMax, YMin, YMax, Confidence, Area]
Index: []

In [76]:
df[df['Area']<=0]

,imageID,LabelName,XMin,XMax,YMin,YMax,Confidence,Area


In [77]:
df.drop(df[df['Area']<=0].index,inplace=True)

In [78]:
df.to_csv('df.csv')